## Our own transformer design
The [Unibalun](https://github.com/DG1JAN/UniBalun) can serve as an example to design our own transformer.
DL2MAN has done some [measurements on this and similar impedance transformers](https://www.youtube.com/watch?v=nNzTf1F12BE).

### Calculation : 
Given parameters:
* source impedance = 50ohm
* turns ratio = 1:7 (will result in 1:49 impedance ratio)
* Max power = 5W
* Frequency range = 3-30MHz

In [8]:
import math
Zin = 50 # [ohm] source impedance
fmin = 3e6 # Lowest usable frequency[Hz]
turns_ratio = 7
Pmax = 5 # [W]

# Calculate impedance of primary coil
# Impedance of primary winding must be greater than 5x source impedance.
Xp_min = 5 * Zin

# Calculate inductance of primary coil
Lp_min = Xp_min / (2*math.pi*fmin)
print('Minimum required inductance is {:.3}H'.format(Lp_min))

Minimum required inductance is 1.33e-05H


### Core materials
* [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials : 
 * [#43](https://www.fair-rite.com/43-material-data-sheet/) < 10MHz, permeability is very temperature dependent
 * [#61](https://www.fair-rite.com/61-material-data-sheet/) < 25MHz, much more stable permeability with respect to temperature.
 * [SIFERRIT K1](https://www.tdk-electronics.tdk.com/download/528860/ec9323ccb94c129f0da1c855d7963b2d/pdf-k1.pdf) : < 20MHz
 * [#67](https://www.fair-rite.com/67-material-data-sheet/) < 50MHz
 
#### Using a #43-core
This core is not suited for this frequency range according to the specs of the core material.  Let's try it anyway because it seems to be so common in the ham HF-world.
 * [Fair-Rite 5943001301](https://www.fair-rite.com/product/toroids-5943001301/)
 * [Fair-Rite 2643625002](https://www.fair-rite.com/product/round-cable-emi-suppression-cores-2643625002/) : Al = 2910nH/N²
 
#### Using 4 W 620-core
 * [WE 7427010](https://www.we-online.com/components/products/datasheet/7427010.pdf)
 
#### Using a #61-core
 * [Amidon FT 82-61  = Fair-rite 5961000601](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961000601/8594131) : ⌀21mm, #61, €1.69
 * [Fair-rite 5961001821](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961001821/15930539) : ⌀23mm, #61, €1.42, ⌀23mm
 
#### Using a #67-core
Let's select a 67-core because it matches the required frequency range: [FT87-67](https://www.amidoncorp.com/ft-87-67/). 

160 windings is too much to wind on such a small FT-67-87 core.  No core has been bought for this frequency range.

 ### Calculations
 * [Miguel VK3CPU - RF Toroid Calculator v1.0](https://miguelvaca.github.io/vk3cpu/toroid.html)- : don't use the wire length calculator from this site as it yields much exxagerated lengths.
 * [Toroid Winding Calculator](http://www.calculatoredge.com/electronics/coilwinding.htm#toroid)

In [9]:
from calcInductors import *

d = 0.5e-3 # [m] wire diameter

# Fair-Rite 5943001301 (#43 material)
# Toroid
Al = 500 # [nH/N²] Inductance factor
Ae = 0.308e-4 # [m²] Effective cross-sectional area
Bmax = 0.22 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials
OD = 25.4e-3 # [m] outer diameter
ID = 15.5e-3 # [m] inner diameter
h = 6.35e-3 # [m] height

# Fair-Rite 5961001821 (#61 material)
# Toroid
# Al = 75 # [nH/N²] Inductance factor
# Ae = 0.262e-4 # [m²] Effective cross-sectional area
# Bmax = 0.25 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials
# OD = 23e-3 # [m] outer diameter
# ID = 12.9e-3 # [m] inner diameter
# h = 7.1e-3 # [m] height

#Amidon FT-87-67 (#67 material)
# Toroid
#Al = 25 # [nH/N²] Inductance factor
#Ae = 0.2612e-4 # [m²] Effective cross-sectional area
#Bmax = 0.23 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials

  
[Np, Ns] = calc_inductor_turns(Al, Lp_min, turns_ratio)
print("Required number of turns on primary winding: " + str(Np))
print("Required number of turns on secondary winding: " + str(Ns))
if Np+Ns < calc_toroid_max_turns(ID, d):
    print("The total number of turns will fit on the inductor.")
else:
    print("The total number of turns doesn't fit on the inductor.")

print("Primary coil wire length: " + str(calc_toroid_wirelength(OD, ID, d, h, Np))+"m")
print("Secondary coil wire length: " + str(calc_toroid_wirelength(OD, ID, d, h, Ns))+"m")

B = calc_max_flux_density(Ae, Pmax, Zin, fmin, Np)
print('Maximum flux density: {:.3}T'.format(B))
if B > Bmax:
    print("Error : Exceeding maximum flux density.")
else:
    print("Flux density OK.")


Required number of turns on primary winding: 6
Required number of turns on secondary winding: 42
The total number of turns will fit on the inductor.
Primary coil wire length: 0.14m
Secondary coil wire length: 0.99m
Maximum flux density: 0.00642T
Flux density OK.


### Winding the core using the "traditional method"

> **NOTE**
>
> * Use enameled Cu-wire.
> * Check if you can actually solder the wire before winding your coils.
> * Use ⌀0.5mm or larger wire.  It keeps its shape more easily and is less likely to form kinks while winding.

1. Cutting 1.2m meter of wire
2. Folding 20cm back on itself and twist.
3. Starting point is where twist ends.  Make 6 windings with the twisted wire.
4. Go back to the starting point.  Make 15 windings with the single wire.
5. Go through the middle and wind back 21 more windings.


### Measurements
If you use Youtube as a source for 49:1 unun measurements, you'll notice that:
1. Most of them only measure S11 (sometimes without even loading the secondary coil).
2. Some of them measure S21 in a back to back measurement : not ideal

#### #43 material

In [10]:
# Given parameters
f = 500e3 # [Hz] Measuring frequency
Z0 = 50 # [Ω] source impedance

# Measured parameters with NanoVNA
Z_open = 1.996 + 52.283j # [Ω] Measured impedance at frequency f of primary coil, with secondary coil open
Z_closed = 0.017 + 4.028j # [Ω] Measured impedance at frequency f of primary coil, with secondary coil closed
SRF = 3.55e6 # [Hz] Open circuit self-resonant frequency
R_max = 442 # [Ω] Open Circuit SRF R_max

# Calculated parameters
transformer_bandwidth_calculation(f, SRF, 7, Z0, Z_open, Z_closed)

Leakage inductance = 1.28e-06H
Magnetization inductance = 1.54e-05H
Al = 3.13e+02 nH/N²
Cw = 1.31e-10 F
Low frequency cut-off (fl) = 5.18e+05Hz
High frequency cut-off due to winding capacitance = 2.43e+07Hz
High frequency cut-off due to resonance of winding capacitance and leakage inductance = 1.23e+07Hz


* [Performance of 49:1 ferrite core transformers](https://squashpractice.com/2021/06/23/performance-of-491-ferrite-core-transformers/)
* [KN5L EFHW Unun](https://www.kn5l.net/kn5lEfhwUnun/)

<img src="./measurements/insertion_loss_.png" alt="Insertion loss versus frequency for various transformer setups" width="2000"/>

Due to wrong calibration, the frequency dependent calibration of mag(S21) is replaced by the theoretical value of 0.04.

#### S21 Measurement using a NanoVNA

See [trafo_matched_load](./trafo_matched_load.ipynb) to find out why a 2400Ω resistor in series with port 2 is needed.

I'm using through hole resistors : 22KΩ in parallel with 2.7kΩ.  That should be 2.40kΩ.  The digital multimeter says 2.35kΩ.  
The NanoVNA software shows a parallel impedance of 2450Ω || 2.79pF

##### Port1 measurement

In [11]:
s11 = 0.95912 # Measured by NanoVNA
Zr_calc = Z0 * (1+s11)/(1-s11)
print('Calculated Zr = {:.3}Ω'.format(Zr_calc))

Calculated Zr = 2.4e+03Ω


So the measurement on port1 looks ok.

##### Port2 measurement

In [12]:
s21 = 0.04 # magnitude value, as measured by NanoVNA.
s21dB = 20*math.log10(s21)
print('s21 = {:.3}dB'.format(s21dB))

s21 = -28.0dB


As expected after calibration.

### Improved design
[Engineering the EFHW 49:1 Transformer and Antenna](https://squashpractice.com/2021/07/20/engineering-the-efhw-491-transformer-and-antenna/)
1. By stacking two toroids, we can get the same inductance with half the number of windings.  Less windings means less capacitive coupling.

In [13]:
# Given parameters
f = 500e3 # [Hz] Measuring frequency
Zs = 50 # [Ω] source impedance

# Measured parameters with NanoVNA
Z_open =  1.206 + 44.270j # [Ω] Measured impedance at frequency f of primary coil, with secondary coil open
Z_closed =  0.087 + 0.618j # [Ω] Measured impedance at frequency f of primary coil, with secondary coil closed
SRF =  6.0e6 # [Hz] Open circuit self-resonant frequency
R_max = 597.538 # [Ω] Open Circuit SRF R_max

# Calculated parameters
transformer_bandwidth_calculation(f, SRF, 4, Zs, Z_open, Z_closed)

Leakage inductance = 1.97e-07H
Magnetization inductance = 1.39e-05H
Al = 8.68e+02 nH/N²
Cw = 5.06e-11 F
Low frequency cut-off (fl) = 5.73e+05Hz
High frequency cut-off due to winding capacitance = 6.29e+07Hz
High frequency cut-off due to resonance of winding capacitance and leakage inductance = 5.04e+07Hz


The measurement results are shown in the same graph (yellow line).
I was expecting better high frequency performance, as the calculated high frequency cut-off values were > 50 MHz.  The secondary winding is very sensitive to capacitive loading.  _Because of the turns ratio N, the secondary capacitance will appear N2 larger at the primary and will resonate with the primary magnetizing inductance. ([Gary Rondeau](https://squashpractice.com/2021/07/20/engineering-the-efhw-491-transformer-and-antenna/))_

#### #61 material
I didn't succeed in making a transformer that fulfilled the specs using this core.

##### Primary winding inductance
Suitable for 5-30MHz.  For 13 windings, the inductance at 3MHz is 13.9µH while it even rises to 23µH at 30MHz.  This behavior matches the µ'-curve of the material permeability versus frequency of the [#61 material datasheet](https://www.fair-rite.com/61-material-data-sheet/).

##### Secondary winding inductance
The secondary coil has a resonance frequency of around 3MHz.  On such a small core, the windings are very close to each other, resulting in a large capacitive coupling and a low self-resonant frequency.
This core is too small for so many windings.

## References
* ARRL Handbook 2018, §5.7.3 Broadband Ferrite RF Transformers
* Secrets of RF Circuit Design, Joseph J. Carr, §4. Winding your own coils
* [Steve Dick, K1RF : The end-fed half-wave antenna](http://gnarc.org/wp-content/uploads/The-End-Fed-Half-Wave-Antenna.pdf)

## Ideas for the future
We could make a transformer, suitable for EFHW and dipole.  Three coils 1:4:5 coil ratio.
* EFHW  = 1 : (4+5) = 1:9
* Dipole = (1+4) : 5 = 5:5